# IAPI Bridge Test

Stand-alone codes using Pythonnet to connect to IAPI

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import glob
import datetime
import time
from pathlib import Path

from IPython import display
import pylab as plt
import numpy as np
import clr
import atexit

In [3]:
import sys
sys.path.append('C:\\Users\\joewa\\Work\\git\\vimms')
sys.path.append('C:\\Users\\Vinny\\work\\vimms')
sys.path.append('C:\\Users\\Orbitrap Fusion\\vimms')

from vimms.Common import *

## Calling the IAPI Bridge from Python

### Load our custom assembly

In [4]:
sys.path.append('C:\\Users\\Orbitrap Fusion\\vimms\\Interface\\iapi_lib')
sys.path.append('C:\\Users\\Orbitrap Fusion\\vimms\\Interface\\iapi_lib\\tribrid')
sys.path.append('C:\\Users\\Orbitrap Fusion\\vimms\\Interface\\FusionBridge\\bin\\Debug')

In [5]:
#ref = clr.AddReference('TestLibraries')
clr.AddReference('FusionBridge')

In [6]:
list(clr.ListAssemblies(True))

['mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089',
 'clrmodule, Version=2.4.0.0, Culture=neutral, PublicKeyToken=null',
 'Python.Runtime, Version=2.4.0.0, Culture=neutral, PublicKeyToken=null',
 'System.Core, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089',
 'System.Configuration, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a',
 'System.Xml, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089',
 'System, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089',
 '__CodeGenerator_Assembly, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null',
 'e__NativeCall_Assembly, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null',
 'FusionBridge, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null',
 'Fusion.API-1.0, Version=1.2.0.0, Culture=neutral, PublicKeyToken=35efc4498449bc4c',
 'API-2.0, Version=1.1.0.1, Culture=neutral, PublicKeyToken=c03bf447005257eb',
 'Spectrum-1.0, Version=1.1.0.1, Cultur

In [7]:
import FusionBridge

In [8]:
FusionBridge.FusionBridge.getIFusionInstrumentAccessContainer()

In [ ]:
ref_dir = 'C:\\Users\\Orbitrap Fusion\\vimms\\Interface\\FusionLibrary\\bin\\Debug\\netstandard2.0'
if ref_dir not in sys.path:
    sys.path.append(ref_dir)

In [ ]:
print(clr.FindAssembly('Fusion'))

In [ ]:
list(clr.ListAssemblies(False))

In [ ]:
ref = clr.AddReference('FusionLibrary')

In [ ]:
list(clr.ListAssemblies(False))

### Create FusionBridge

Below codes generate a bunch of custom scans

In [ ]:
from FusionLibrary import FusionBridge

In [ ]:
filename = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Data\\multibeers_urine_data\\beers\\fragmentation\\mzML\\Beer_multibeers_1_T10_POS.mzML'
filename = None # to connect to the actual machine

In [ ]:
def scan_handler(scan):
    print('Received scan %s -- %d peaks' % (scan.Header['Scan'], scan.CentroidCount))
    
def state_changed_handler(state):
    print('state_change_handler is called')

class CustomScanHandler(object):
    def __init__(self, fusion_bridge, start_mz, end_mz, isolation_width, collision_energy, ms_level, polarity, first_mass, last_mass, single_processing_delay):
        self.start_mz = start_mz
        self.end_mz = end_mz
        self.isolation_width = isolation_width
        self.collision_energy = collision_energy
        self.ms_level = ms_level
        self.polarity = polarity
        self.first_mass = first_mass
        self.last_mass = last_mass
        self.single_processing_delay = single_processing_delay        
        self.precursor_mass = self.start_mz
        self.fusion_bridge = fusion_bridge
        self.running_number = 100000
            
    def step(self):
        if self.precursor_mass < self.end_mz:
            print('custom_scan_handler called for precursor_mass %f' % self.precursor_mass)
            self.fusion_bridge.CreateCustomScan(self.running_number, self.precursor_mass, self.isolation_width, self.collision_energy, self.ms_level, self.polarity, 
                                                self.first_mass, self.last_mass, self.single_processing_delay)
            self.precursor_mass += 0.02
            self.running_number += 1

In [ ]:
start_mz = 524.0
end_mz = 525.0

# custom scan parameters
isolation_width = 0.7
collision_energy = 35.0
ms_level = 2
polarity = POSITIVE
first_mass = 50.0
last_mass = 600.0
single_processing_delay = 0.50

In [ ]:
show_console_logs = True
log_dir='C:\\Xcalibur\\data\\Joe\\logs'
fusion_bridge = FusionBridge(filename, log_dir, show_console_logs)
atexit.register(fusion_bridge.CloseDown)

# create event handlers
scan_handler_delegate = FusionBridge.UserScanArriveDelegate(scan_handler)
state_changed_delegate = FusionBridge.UserStateChangedDelegate(state_changed_handler)
scan_handler = CustomScanHandler(fusion_bridge, start_mz, end_mz, isolation_width, collision_energy, ms_level, polarity, first_mass, last_mass, single_processing_delay)
custom_scan_delegate = FusionBridge.UserCreateCustomScanDelegate(scan_handler.step)

fusion_bridge.SetEventHandlers(scan_handler_delegate, state_changed_delegate, custom_scan_delegate)
fusion_bridge.CreateCustomScan(scan_handler.running_number, scan_handler.precursor_mass, isolation_width, collision_energy, ms_level, polarity, first_mass, last_mass, single_processing_delay)

In [ ]:
fusion_bridge.CloseDown()